# 1. Thư viện

In [1]:
import os
import json
import re
from docx import Document

# 2. Thực thi

## 2.1. Hàm đếm từ

In [2]:
def count_tokens_simple(text):
    # Đếm số tokens đơn giản dựa trên khoảng trắng
    return len(text.split())

## 2.2. Hàm trích xuất các từ khóa

In [3]:
def extract_keywords(text):
    # Loại bỏ dấu câu ngoại trừ "/" và "-"
    text = re.sub(r'[^\w\s/-]', '', text)
    words = text.split()
    unique_words = list(dict.fromkeys(words))  # Loại bỏ từ trùng lặp
    return unique_words

In [4]:
def process_keywords(keywords):
    result = []
    
    # Biểu thức chính quy để nhận diện số La Mã
    roman_numeral_pattern = re.compile(r"^(?=[MDCLXVI])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$")
    
    for word in keywords:
        if roman_numeral_pattern.match(word) or any(c in word for c in "/-"):  # Giữ nguyên số La Mã hoặc chứa "/" hay "-"
            result.append(word)
        else:
            result.append(word.lower())  # Chuyển các từ còn lại thành chữ thường

    return result

## 2.3. Hàm xử lý tên file

In [5]:
def process_file_name(file_name):
    # Giả sử số thứ tự nằm ở phần đầu tên file và được tách ra bằng dấu "_"
    parts = file_name.replace('.docx', '').split('_')

    if len(parts) != 8:
        return None  # Nếu file không hợp lệ
    
    stt = parts[0]  # Số thứ tự
    loai_van_ban = parts[1]
    noi_ban_hanh = parts[2]
    so_hieu = parts[3].replace("-", "/")
    linhvuc_nganh = parts[4]
    ngay_ban_hanh = parts[5].replace("-", "/")
    ngay_hieu_luc = parts[6] if parts[6] == "Đã biết" else parts[6].replace("-", "/")
    chu_de = parts[7]
    
    # Tách keywords từ tên file và xử lý
    metadata_keywords = extract_keywords(f"{loai_van_ban} {noi_ban_hanh} {so_hieu} {linhvuc_nganh} {ngay_ban_hanh} {ngay_hieu_luc} {chu_de}")
    metadata_keywords = process_keywords(metadata_keywords)

    return {
        "stt": stt,  # Thêm số thứ tự
        "loai_van_ban": loai_van_ban,
        "noi_ban_hanh": noi_ban_hanh,
        "so_hieu": so_hieu,
        "linhvuc_nganh": linhvuc_nganh,
        "ngay_ban_hanh": ngay_ban_hanh,
        "ngay_hieu_luc": ngay_hieu_luc,
        "chu_de": chu_de,
        "key_words": metadata_keywords  # Lưu lại key words từ tên file
    }

## 2.4. Hàm đọc và xử lý nội dung của file .docx

In [6]:
def process_docx(file_path, file_metadata, max_tokens):
    document = Document(file_path)

    current_chapter = None
    current_section = None
    current_mini_section = None
    data = []
    
    for i in range(len(document.paragraphs)):
        paragraph_text = document.paragraphs[i].text.strip()

        if paragraph_text.startswith("Chương"):
            current_chapter = paragraph_text
            current_section = None
            current_mini_section = None
            if i + 1 < len(document.paragraphs):
                next_paragraph = document.paragraphs[i + 1].text.strip()
                if not next_paragraph.startswith(("Mục", "Tiểu mục", "Điều")):
                    current_chapter += f": {next_paragraph}"
            continue

        if paragraph_text.startswith("Mục"):
            current_section = paragraph_text
            current_mini_section = None
            continue

        if paragraph_text.startswith("Tiểu mục"):
            current_mini_section = paragraph_text
            continue

        if paragraph_text.startswith("Điều"):
            article = paragraph_text
            content = []
            article_sections = []  # Lưu nội dung các Khoản

            for j in range(i + 1, len(document.paragraphs)):
                next_paragraph = document.paragraphs[j].text.strip()
                if next_paragraph.startswith(("Chương", "Mục", "Tiểu mục", "Điều")):
                    break
                content.append(next_paragraph)

                # Kiểm tra xem đoạn văn có phải là Khoản không
                if re.match(r'^\d+\.', next_paragraph):
                    article_sections.append(next_paragraph)

            # Lưu trữ nội dung không phải Khoản
            content_text = "\n".join(content)
            
            # Tách từ khóa từ nội dung và gộp với từ khóa từ file_name
            content_key_words = process_keywords(extract_keywords(f"{article}\n{content_text}"))
            
            # Gộp từ khóa từ file name và nội dung rồi loại bỏ trùng lặp
            all_key_words = list(dict.fromkeys(file_metadata["key_words"] + content_key_words))

            # Tạo thêm từ khóa mới nếu có ký tự "/" trong từ khóa
            for kw in all_key_words[:]:  # Sử dụng [:] để sao chép danh sách trong vòng lặp
                if "/" in kw:
                    new_kw = kw.replace("/", "-")
                    if new_kw not in all_key_words:  # Kiểm tra xem từ khóa mới đã có chưa
                        all_key_words.append(new_kw)

            # Kiểm tra số lượng tokens
            if count_tokens_simple(content_text) > max_tokens - count_tokens_simple(article):
                numbered_sections = re.split(r'(?=\d+\.\s)', content_text)

                current_content = ""
                for section in numbered_sections:
                    if count_tokens_simple(current_content + section) <= max_tokens - count_tokens_simple(article):
                        current_content += section + "\n"
                    else:
                        data.append({
                            "Chapter": current_chapter,
                            "Section": current_section if current_section else None,
                            "Mini-Section": current_mini_section if current_mini_section else None,
                            "Article": article,
                            "Content": current_content.strip(),
                            "Article-Section": f"Khoản {len(article_sections) + 1}.",  # Lưu số thứ tự Khoản
                            "combine_Article_Content": f"{article}\n{current_content.strip()}",
                            "key_words": all_key_words
                        })
                        current_content = section + "\n"

                if current_content:
                    data.append({
                        "Chapter": current_chapter,
                        "Section": current_section if current_section else None,
                        "Mini-Section": current_mini_section if current_mini_section else None,
                        "Article": article,
                        "Content": current_content.strip(),
                        "Article-Section": f"Khoản {len(article_sections) + 1}.",  # Lưu số thứ tự Khoản
                        "combine_Article_Content": f"{article}\n{current_content.strip()}",
                        "key_words": all_key_words
                    })
            else:
                # Nếu không có Khoản
                if article_sections:
                    for section in article_sections:
                        data.append({
                            "Chapter": current_chapter,
                            "Section": current_section if current_section else None,
                            "Mini-Section": current_mini_section if current_mini_section else None,
                            "Article": article,
                            "Content": section.strip(),
                            "Article-Section": section.split('.')[0] + '.',  # Lưu số thứ tự Khoản
                            "combine_Article_Content": f"{article}\n{section.strip()}",
                            "key_words": all_key_words
                        })
                else:
                    data.append({
                        "Chapter": current_chapter,
                        "Section": current_section if current_section else None,
                        "Mini-Section": current_mini_section if current_mini_section else None,
                        "Article": article,
                        "Content": content_text.strip(),
                        "Article-Section": None,
                        "combine_Article_Content": f"{article}\n{content_text.strip()}",
                        "key_words": all_key_words
                    })

    return data

## 2.5. Hàm chính xử lý tất cả các file trong thư mục

In [7]:
def process_folder(folder_path, output_json_path, max_tokens):
    all_data = []
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".docx"):
            file_metadata = process_file_name(file_name)
            if file_metadata is None:
                continue
            
            file_path = os.path.join(folder_path, file_name)
            docx_data = process_docx(file_path, file_metadata, max_tokens=max_tokens)
            
            for entry in docx_data:
                combined_entry = {**file_metadata, **entry}
                all_data.append(combined_entry)
    
    with open(output_json_path, 'w', encoding='utf-8') as json_file:
        json.dump(all_data, json_file, ensure_ascii=False, indent=4)
    
    print(f"Dữ liệu đã được lưu vào file {output_json_path}")

## 2.6. Thực thi

In [8]:
# Đường dẫn tới thư mục và file đầu ra
folder_path = r'data\trich_dan_luat\docx\Luat'
json_file_path = r'data\trich_dan_luat\json\test_Combined_Output_Token_Keywords.json'
max_tokens = 2048  # Giới hạn số token

In [9]:
# Chạy chương trình chính
process_folder(folder_path, json_file_path, max_tokens)

Dữ liệu đã được lưu vào file data\trich_dan_luat\json\test_Combined_Output_Token_Keywords.json
